In [352]:
# converter of geochemical data for "endmembers" of specific to GHG rocks
# author:  thomas kosciuch 
# contact: thomas@kosciuch.ca
# affilite: University of Toronto - Earth Sciecnes


# 1. Directory Structure
#
#   python code
#           /XSLX/
#               xlsx files
# 
import scipy #dtypes
import numpy as np #dtypes
import pandas as pd
import os as os            #checks if directories exist for file writing
from os import listdir
import mysql.connector
import sqlite3
import MySQLdb

#converts to CSV
import xlrd
import csv



####################################
##  finds files in source folder  ##
folder = "XLSX"                    #
files = listdir(folder)            #
path = folder + "/" + files[0]     #
file = pd.ExcelFile(path)          #
sheets = file.sheet_names          #
                                  ## 
# loading:                     #####
######## Convert to CSV: ###########
destination_folder = "csv"
if os.path.isdir(destination_folder) == False:
    mkdir(destination_folder)

# i had to use no header, because some are unnamed
notKnown = pd.read_excel(file,sheets[0], header=None, keep_default_na=True)
csvName = destination_folder + '/' + sheets[0] +'.csv'
notKnown.to_csv(csvName, encoding='utf-8')
notKnown = pd.read_csv(csvName, na_values="NaN")

minerals = pd.read_excel(file,sheets[1],  keep_default_na=True)
csvName = destination_folder + '/' + sheets[1] +'.csv'
minerals.to_csv(csvName, encoding='utf-8')
minerals = pd.read_csv(csvName, na_values="NaN")

thermody = pd.read_excel(file,sheets[2], header=None, keep_default_na=True)
csvName = destination_folder + '/' + sheets[2] +'.csv'
thermody.to_csv(csvName, encoding='utf-8')
thermody = pd.read_csv(csvName, na_values="NaN")
###################################
# removed unnamed row in minerals #
# minerals = minerals.loc[:, ~minerals.columns.str.contains('^Unnamed')]

In [353]:
# Index
# minerals.loc[[54]]

# first we do manual entry, then we iterate:
print("how many minerals?")
minNum = int(input())
mins = [0] * minNum
for i in range(minNum):
    print("what is the No. of mineral ",+(i+1),"?")    
    mins[i] = int(input())

#converting to number sys in Excel
mins = [x-1 for x in mins]


how many minerals?
4
what is the No. of mineral  1 ?
57
what is the No. of mineral  2 ?
138
what is the No. of mineral  3 ?
143
what is the No. of mineral  4 ?
105


In [446]:
minNum = 4
mins = [57, 138, 143, 105]
mins = [x-1 for x in mins]

########### Creates definition that uploads relevant minerals
d={} # set up empty dictionary
# uploads water first: 

endmember = [0] * ( minNum + 2)
endmember[minNum]   = 151
endmember[minNum+1] = 152

for i in range(minNum):
    endmember[i]     = mins[i]
    d["Mineral_{0}".format(i)] = {
        "minrl"      :  str(minerals.loc[mins[i],"min_sym"]),
        "min_name"   : str(minerals.loc[mins[i],"mineral_name"]),
        "sys"        : "",
        "endmember"  : [mins[i]],
        "endnum"     : 2,
        "variables"  : [],
        "vari_lower" : [],
        "vari_upper" : [],
        "p"          : "@(X)X(1)",
        "SiO2toCO2"  : minerals.loc[mins[i],"SiO2":"CO2"]
        }
# Add water
    d["Mineral_{0}".format((minNum))] = {
    "minrl"      : "liq",
    "name"       : "water",
    "sys"        : "CHO",
    "endmember"  : [151],
    "endnum"     : 2,
    "variables"  : {'x(liq)'},
    "vari_lower" : [0],
    "vari_upper" : [1],
    "p"          : "@(X)X(1)",
    "SiO2toCO2"  : minerals.loc[150,"SiO2":"CO2"]
}
# Add CO2
d["Mineral_{0}".format((minNum+1))] = {
    "minrl"      : "liq",
    "name"       : "CO2",
    "sys"        : "CHO",
    "endmember"  : [152],
    "endnum"     : 2,
    "variables"  : {'x(liq)'},
    "vari_lower" : [0],
    "vari_upper" : [1],
    "p"          : "@(X)X(1)",
    "SiO2toCO2"  : minerals.loc[151,"SiO2":"CO2"]
}

#############################################################

In [444]:
# del d
#print(minerals.loc[[54]])
# C = minerals.loc[mins[i],"SiO2":"CO2"]
#print(d)


In [513]:
# Starting variables
T_min   = 200
T_max   = 1000
P       = 2                         # [kbar]
res     = 101                       # resolution

####################
TRange  = [T_min, T_max]
XCO2    = np.linspace(0,0.5, res)   # sets up plotting 
XCO2    = XCO2[1:res]               # removes final CO2 point
Phase   = mins                      # Sets up "Phase" variable to mimic MatLab
T       = [0] * res                 # creates empty matrix
# First guess for temperature
T_guess = (TRange[0]+TRange[1])/2   # guesses halfway between range
Xs      = [] 
CC=['  SiO2   ','  TiO2   ',' Al2O3   ','   FeO   ','   MgO   ','   MnO   ','   CaO   ','  Na2O   ','   K2O   ','     O   ','   H2O   ','   CO2   ',' Supl1   ',' Supl2   ']


In [536]:
# CALC UNI T AT P  - ITERATIVE
# Counts the number of 'end-members' each phase has

PHASE         = d
endnum        = np.zeros(minNum+2)
for i in range(0, minNum+2):
    target    = 'Mineral_' +  str(i)
    temp      = d.get(target)
    endnum[i] = len(temp.get('endmember'))

In [533]:
# NOW function [React,A0,C]=CalcMatrix(phase)
"""% construct linear metamorphic reactions with the input phases.
% =======================================
% phase is the data-structure for solid solution models.
% A0 is the composition matrix before Gaussian elimination
% C is the Modal composition"""

# setup empty matrix (reserve memory)
A = np.zeros(((len(PHASE)), len(CC)))

# Compositional matrix
# Col 1: 15 = CC
for i in range((len(PHASE))):
    # -2 because of the 2 supplemental columns
    for j in range(len(CC)-2):                 
        target        = 'Mineral_' +  str(i)
        temp          = d.get(target)
        temp          = temp.get('SiO2toCO2')
        A[i][j]       = int(temp[j])

# Deletes zero columns
components = []
for i in range(len(CC)-1):
    target = len(CC) - 1 - i
    if sum(A[:,target]) > 0:               # Preserves present components
        components.insert(0,(CC[target]))    
    if sum(A[:,target]) == 0:
        A = np.delete(A,target,1)

A = [*zip(*A)]                             # Transpose. I think this helps for solving
A0 = A                                     # Copies A


In [545]:
A

[(8.0, 0.0, 0.0, 1.0, 0.0, 0.0),
 (5.0, 0.0, 1.0, 0.0, 0.0, 0.0),
 (2.0, 1.0, 1.0, 0.0, 0.0, 0.0),
 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0),
 (0.0, 1.0, 2.0, 0.0, 0.0, 1.0)]

In [ ]:
if(dtype(agg[y]) == 'string'):

In [28]:
import sys

con = None

try:
    con = sqlite3.connect('test.db')

    cur = con.cursor()
    cur.execute('SELECT SQLITE_VERSION()')

    data = cur.fetchone()

    print "SQLite version: {}".format(data)

except sqlite3.Error, e:

    print "Error {}:".format(e.args[0])
    sys.exit(1)

finally:

    if con:
        con.close()  
                     

SyntaxError: invalid syntax (<ipython-input-28-9cb66b75d9f3>, line 13)

In [6]:
# 1. create sql struct per file

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()


In [ ]:
conn = sqlite3.connect('mineralEndmembers.db')
c = conn.cursor()
# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()
conn.close()

In [ ]:
variable = ["a", "b"]
len(variable)

In [ ]:
# f= open("guru99.txt","w+")